<a href="https://colab.research.google.com/github/a3d2ddk/post-collision-trajectory-estimation-from-impact-pose/blob/main/collision_trajectory_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

if os.path.exists('/content/repo'):
  print('Repository already cloned!')
else:
  print('Cloning repository...')
  !git clone --quiet https://github.com/a3d2ddk/post-collision-trajectory-estimation-from-impact-pose.git
  !mv post-collision-trajectory-estimation-from-impact-pose repo
  print('Repository cloned!')

Cloning repository...
Repository cloned!


In [2]:
import os, glob

from google.colab import drive
drive.mount('/content/drive')

need_pytorch3d=False

try:
  import pytorch3d
except ModuleNotFoundError:
  eed_pytorch3d=True
if need_pytorch3d:
  os.chdir('/content/drive/MyDrive/pytorch3d_wheels')

  wheel_files = glob.glob('*.whl')
  wheel = max(wheel_files, key=os.path.getmtime)
  print("Latest created wheel:", wheel)

  # We try to install PyTorch3D from repo wheel.
  !pip install {wheel}
  #!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  print('Packages installed!')

Mounted at /content/drive


In [13]:
import os, glob
import numpy as np
from PIL import Image

dataset_dir = '/content/repo/Datasets/run_0001'
frame_dir = dataset_dir + '/rgb'
seg_dir = dataset_dir + '/instance_seg'

num_frames = 180

frames = []
segs = []

for i in range(num_frames):
  str_num = str(i).zfill(4)

  frames.append(np.array(Image.open(frame_dir + '/rgb_' + str_num + '.png')))
  seg_i = np.load((seg_dir + '/instance_seg_' + str_num + '.npy'), allow_pickle=True).item()
  segs.append(seg_i['data'])

#print(frames[0].shape)
#print(segs[0].shape)

(1024, 1024)


In [28]:
print(segs[0].shape)

(1024, 1024)


In [33]:
#non_zero_count = np.count_nonzero(segs[0] == 2)
#print(non_zero_count)

starting_img = []
final_cube1 = []
final_cube2 = []

zs = np.zeros_like(frames[0])

for i in range(num_frames):
  fin_img = zs
  starting_img = frames[i]

  fin_img[segs[i] == 2][:] = starting_img[segs[i] == 2][:]
  final_cube1.append(fin_img)

  fin_img[segs[i] == 3][:] = starting_img[segs[i] == 3][:]
  final_cube2.append(fin_img)

print(final_cube1[0].shape)

(1024, 1024, 3)
